In [94]:
import pandas
import numpy as np
import os


In [104]:
data_path = './data/Participant Tracked Data'
total_df = pandas.DataFrame()
# Get every single participant spreadsheet
for task in next(os.walk(data_path))[1]:
    path = data_path + '/' + task
    for filename in os.listdir(path):
        try:
            df = pandas.read_excel(f"{path}/{filename}", header= 2)
        except Exception as e: 
            print(f"{path}/{filename}")
            print(e)
        try: 
            # Append them to the big dataframe after transforming them to match columns
            df = df.iloc[:,:4]
            if list(df.columns) == ['Serial', 'Sentence', 'Objective Rating', 'Far Away Rating']:
                df = df.astype({
                    'Serial': 'int64',
                    'Sentence': 'object',
                    'Objective Rating': 'float64',
                    'Far Away Rating': 'float64'
                })
                total_df = total_df.append(df)
            else:
                print(f"DF READ: {path}/{filename}")
                print(df)
        except (Exception) as e:
            print(f"{path}/{filename}")
            print(e)

            

In [96]:

averages = total_df.groupby('Sentence', as_index=False).mean()[['Sentence', 'Objective Rating', 'Far Away Rating']]
counts= total_df.groupby('Sentence', as_index=False).count()[['Sentence', 'Serial']]
counts.columns = ['Sentence', 'Count']


info =averages.merge(counts,  on= 'Sentence')
info = info[1:].reset_index()
info

index                                           Sentence  \
0         1  (When Alabama seceded from the Union on Januar...   
1         2  -and she said you taught me all wrong, so we c...   
2         3  -until you climb into his skin and walk around...   
3         4  A Hot Steams somebody who cant get to heaven, ...   
4         5  A Negro would not pass the Radley Place at nig...   
5         6      A PSed photo. It must be the case. IT's fake.   
6         7  A basebal hit into the Radley yard was a lost ...   
7         8                            A beautiful young girl.   
8         9  A bit sad. this man seems to be in a lot of pa...   
9        10  A child carelessly hurt his head, and is in a ...   
10       11             A child was tortured and is desperate.   
11       12  A cute baby is smiling. He looks like he's hav...   
12       13  A cute baby looks surprised. She's wearing an ...   
13       14  A day was twenty-four hours long but seemed lo...   
14       15  A feel the boys uncomfort. I hope he gets out ...   
15       16  A girl standing. she appears kind of confusrd ...   
16       17  A good home remedy forBurris, I want you to go...   
17       18  A haint lives there, he said cordially, pointi...   
18       19  A kid is crying? But that's only because of a ...   
19       20  A kid who has come to the world too early. But...   
20       21                        A kind old man from Africa.   
21       22  A man is smiling. I'm not sure what he's smili...   
22       23  A person's hand may have been cut off or somet...   
23       24           A scene in movie to show a murder case.    
24       25  A storm of laughter broke loose when it finall...   
25       26  A stylish phone call? Interesting. A cosy home...   
26       27  A teenager looks kind of pissed that she has t...   
27       28  A terrible disease has struck this person?Is t...   
28       29  A tiny, almost invisible movement, and the hou...   
29       30  A woman carelessly hurt herself, her husband w...   
...     ...                                                ...   
1851   1852  this person looks unhappy and seems to have an...   
1852   1853  this person may be permanently disfigured? i c...   
1853   1854  this picture is of someone in the middle of su...   
1854   1855  this woman died in a fire and is now being exa...   
1855   1856  this woman has her face ripped up what happene...   
1856   1857  this woman looks like she's in an abusive rela...   
1857   1858           upset that he's been abused and worries.   
1858   1859  very bloody. what is in his mouth. blood looks...   
1859   1860  very old picture. man is sunburned or graphics...   
1860   1861  very red face. is this guy martian. man just c...   
1861   1862  very wise man. years of wisdom and experienced...   
1862   1863  weird throwback picture makes me kid of feel w...   
1863   1864  what happened to their nose this is sad and up...   
1864   1865  what in the world happened to that hand? it mi...   
1865   1866  what is happening to their hand why is there s...   
1866   1867  what? a halloween set. white head? i see 3 tee...   
1867   1868  wheres the eyeball hole in nose. teeth are mes...   
1868   1869  why is he covered in blood but does not look l...   
1869   1870  woah he's bloody but he's lying horizontal so ...   
1870   1871  woah that's some nice makeup there, he's defin...   
1871   1872  woman got hit in the eye but shes fierce and i...   
1872   1873  worried for the babies health but hopeful beca...   
1873   1874  wow, surprised , star?struck that's me right n...   
1874   1875  yikes this is a strange thing but looks seriou...   
1875   1876  yikes this makes me feel uncomfortable and it ...   
1876   1877  young beautiful lady sitting on a beach in vac...   
1877   1878              young beautiful woman being tortured.   
1878   1879         young gentleman is going to say something.   
1879   1880                                   

In [168]:
# Check if info shares any data w/ test data
test_df = pandas.read_excel('./deprecate/Proofread_LDH_Data_Readable_Test.xlsx')
test_df.columns = ['Sentence', 'Objective Rating', 'Far Away Rating']

interceptdf = pandas.merge(test_df, info, how='inner', on='Sentence')
interceptdf['Objective Rating_y'] = (interceptdf['Objective Rating_y'] * interceptdf['Count'] + interceptdf['Objective Rating_x'])/ (interceptdf['Count'] + 1)
interceptdf['Far Away Rating_y'] = (interceptdf['Far Away Rating_y'] *interceptdf['Count'] + interceptdf['Far Away Rating_x'])/ (interceptdf['Count'] + 1)

interceptdf = interceptdf[['Sentence', 'Objective Rating_y', 'Far Away Rating_y']]
interceptdf.columns = ['Sentence', 'Objective Rating', 'Far Away Rating']

# # Data with overlap becomes test data
interceptdf.to_excel('./test/test_data.xlsx')




In [171]:
# Get data that doesn't overlap: that data becomes training data
all_data = info.append(test_df)
all_data = all_data.groupby('Sentence').filter(lambda x: len(x) < 2)[['Sentence', 'Objective Rating', 'Far Away Rating']]
# all_data = all_data.reset_index()
# all_data.rename(columns = {'index': 'Serial'}, inplace= True)


training_data = all_data[['Sentence', 'Objective Rating', 'Far Away Rating']]
# training_data.to_excel('./training/training_data.xlsx')

In [172]:
##Confirm no overlap between training and test data

check = pandas.merge(interceptdf, training_data, how='inner', on='Sentence')
check

Empty DataFrame
Columns: [Sentence, Objective Rating_x, Far Away Rating_x, Objective Rating_y, Far Away Rating_y]
Index: []